# Exploratory Notebook

In [15]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('../Data/im.db')



In [16]:
df_mb = pd.read_sql("""

SELECT *
FROM movie_basics







""", conn)

In [17]:
df_mb.columns

Index(['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres'],
      dtype='object')

In [18]:
df_mb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [19]:
df_mb['genres'].isna().sum()

5408

In [21]:
columns_to_drop_na = ['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres']
df_mb.dropna(subset=columns_to_drop_na, inplace=True)


In [22]:
df_mb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112232 entries, 0 to 146139
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         112232 non-null  object 
 1   primary_title    112232 non-null  object 
 2   original_title   112232 non-null  object 
 3   start_year       112232 non-null  int64  
 4   runtime_minutes  112232 non-null  float64
 5   genres           112232 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.0+ MB


In [23]:
df_mb['genres'].isna().sum()

0

In [24]:
df_mr = pd.read_sql("""

SELECT *
FROM movie_ratings







""", conn)

In [25]:
df_mr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [26]:
df_mr.columns

Index(['movie_id', 'averagerating', 'numvotes'], dtype='object')

In [27]:
columns_to_drop_na = ['movie_id', 'averagerating', 'numvotes']
df_mr.dropna(subset=columns_to_drop_na, inplace=True)
df_mr.isna().sum()

movie_id         0
averagerating    0
numvotes         0
dtype: int64

In [28]:
df_mr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.3+ MB


In [29]:
df_ma = pd.read_sql("""

SELECT *
FROM movie_akas


""", conn)

In [30]:
df_ma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   movie_id           331703 non-null  object 
 1   ordering           331703 non-null  int64  
 2   title              331703 non-null  object 
 3   region             278410 non-null  object 
 4   language           41715 non-null   object 
 5   types              168447 non-null  object 
 6   attributes         14925 non-null   object 
 7   is_original_title  331678 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 20.2+ MB


In [31]:
df_ma.columns

Index(['movie_id', 'ordering', 'title', 'region', 'language', 'types',
       'attributes', 'is_original_title'],
      dtype='object')

In [ ]:
#columns_to_drop_na = ['movie_id', 'ordering', 'title', 'region', 'language', 'types',
       'attributes', 'is_original_title']
#df_ma.dropna(subset=columns_to_drop_na, inplace=True)

In [32]:
df_ma.isna().sum()


movie_id                  0
ordering                  0
title                     0
region                53293
language             289988
types                163256
attributes           316778
is_original_title        25
dtype: int64

In [74]:
df_5 = pd.read_sql("""

SELECT movie_id, primary_title, genres, MAX (averagerating) AS av_rating , MAX(numvotes) AS num_votes
FROM movie_basics
    JOIN movie_ratings 
       USING(movie_id)
        GROUP BY movie_id
        ORDER BY num_votes DESC;



""", conn)

In [75]:
df_5.head(10)

,movie_id,primary_title,genres,av_rating,num_votes
0,tt1375666,Inception,"Action,Adventure,Sci-Fi",8.8,1841066
1,tt1345836,The Dark Knight Rises,"Action,Thriller",8.4,1387769
2,tt0816692,Interstellar,"Adventure,Drama,Sci-Fi",8.6,1299334
3,tt1853728,Django Unchained,"Drama,Western",8.4,1211405
4,tt0848228,The Avengers,"Action,Adventure,Sci-Fi",8.1,1183655
5,tt0993846,The Wolf of Wall Street,"Biography,Crime,Drama",8.2,1035358
6,tt1130884,Shutter Island,"Mystery,Thriller",8.1,1005960
7,tt2015381,Guardians of the Galaxy,"Action,Adventure,Comedy",8.1,948394
8,tt1431045,Deadpool,"Action,Adventure,Comedy",8.0,820847
9,tt1392170,The Hunger Games,"Action,Adventure,Sci-Fi",7.2,795227


In [ ]:
df_5 = pd.read_sql("""

SELECT movie_id, primary_title, genres, MAX (averagerating) AS av_rating , MAX(numvotes) AS num_votes
FROM movie_basics
    JOIN movie_ratings 
       USING(movie_id)
        GROUP BY movie_id
        ORDER BY num_votes DESC;



""", conn)